<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
(ds_train, ds_test), ds_info = tfds.load('cifar100', split = ['train', 'test'], shuffle_files=True,
                             as_supervised = True, with_info = True, batch_size=16)

In [25]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

In [5]:
# Create a subset of ds_train and ds_val
ds_train_subset = ds_train.take(500)
ds_val = ds_train.skip(600).take(50)

# Convert datasets to NumPy arrays
def dataset_to_numpy(dataset):
    features = []
    labels = []

    for feature, label in tfds.as_numpy(dataset):
        features.append(feature)
        labels.append(label)

    return np.array(features), np.array(labels)

# Convert ds_train_subset and ds_val to NumPy arrays
np_train_features, np_train_labels = dataset_to_numpy(ds_train_subset)
np_val_features, np_val_labels = dataset_to_numpy(ds_val)

In [6]:
print("-------TRAIN--------")
print(np_train_features.shape)
print(np_train_labels.shape)
print("-------VALIDATION--------")
print(np_val_features.shape)
print(np_val_labels.shape)

-------TRAIN--------
(500, 16, 32, 32, 3)
(500, 16)
-------VALIDATION--------
(50, 16, 32, 32, 3)
(50, 16)


In [7]:
np_test_features, np_test_labels = dataset_to_numpy(ds_test)
print("-------TEST--------")
print(np_test_features.shape)
print(np_test_labels.shape)

-------TEST--------
(625, 16, 32, 32, 3)
(625, 16)


In [8]:
CLASS_NUM = np_test_labels.shape[1]
print(CLASS_NUM)

16


In [21]:
tf.keras.backend.clear_session()
model = Sequential([
    Input(shape = (32, 32, 3), dtype = 'float64'),
    Conv2D(16, 3, padding = 'valid', activation = 'relu'),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Conv2D(16, 3, padding = 'valid', activation = 'relu'),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Flatten(),

    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),

    Dense(CLASS_NUM, activation = 'softmax')

    ])
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], run_eagerly=True)

In [22]:
model.fit(ds_train, epochs = 3)

Epoch 1/3
2194/3125 [====================>.........] - ETA: 41s - loss: nan - accuracy: 0.0102

KeyboardInterrupt: 